In [1]:
from chords_and_notes_dict import * 
import threading 
import tkinter as tk
from PIL import ImageTk, Image
import mido
import random 
import os
from functools import partial
import pickle
import time
from datetime import datetime
import copy
import rtmidi as rt

# Script

In [2]:
def open_port(input_name: str, output_name: str): 
    """ 
    Input_port: MIDI controller to send MIDI messages 
    Output_port: Corresponds to the sound that the programm returns after getting MIDI message --> it can be no sound 

    Case 1: If input and output ports are available --> open them with the names given in parameters
    Case 2: If input and output ports are already used --> close them and open them with the names given in parameters

    It returns the input_port and output_port 
    """

    global input_port, output_port

    #if input_port and output_port are AVAILABLE
    try: 
        input_port = mido.open_input(input_name)
        output_port = mido.open_output(output_name)

    #if input_port and output_port are NOT AVAILABLE    
    except Exception:
        close_port(input_port, output_port)
        input_port = mido.open_input(input_name)
        output_port = mido.open_output(output_name)
    
    return input_port, output_port 


def close_port(input_port, output_port) -> None: 
    """ 
    Close input and output ports
    """
    input_port.close()
    output_port.close()


def choose_ports():
    """ 
    Create a Tkinter window in which the input and output names are displayed on your machine
    
    - Select ONLY one for the input by clicking inside a checkbox in the INPUTS column
    - Select ONLY one for the ouput by clicking inside a checkbox in the OUTPUTS column

    It returns the input_name and output_name selected
    """

    def validate_checkboxes():
        """ 
        Retrieve the names of the input and output selected by clicking inside their checkbox
        Then it kills the Tkinter "Inputs & Outputs" window
        """

        global selected_input, selected_output

        input_index = [var.get() for var in input_vars].index(1)
        selected_input = input_options[input_index]
        output_index = [var.get() for var in output_vars].index(1)
        selected_output = output_options[output_index]

        print(f"Input selected: '{selected_input}'")
        print(f"Output selected: '{selected_output}'")

        window.destroy()


    window = tk.Tk()
    window.geometry("600x400")

    input_label = tk.Label(window, text="INPUTS", font=("Aerial", 20))
    input_label.grid(row=0, column=0, sticky="w")

    output_label = tk.Label(window, text="OUTPUTS", font=("Aerial", 20))
    output_label.grid(row=0, column=1, sticky="w")

    # Create the lists of input and output options available on your machine
    input_options = mido.get_input_names()
    output_options = mido.get_output_names()

    # Create variables to track the state of the checkboxes
    input_vars = []
    output_vars = []

    # Create checkboxes for the input options
    for i, option in enumerate(input_options):
        var = tk.IntVar()
        checkbox = tk.Checkbutton(window, text=option, variable=var)
        checkbox.grid(row=i+1, column=0, sticky="w")
        input_vars.append(var)

    # Create checkboxes for the output options
    for i, option in enumerate(output_options):
        var = tk.IntVar()
        checkbox = tk.Checkbutton(window, text=option, variable=var)
        checkbox.grid(row=i+1, column=1, sticky="w")
        output_vars.append(var)

    # Create the entry button to validate the checkboxes
    button = tk.Button(window, text="Validate", command=validate_checkboxes)
    button.grid(row=max(len(input_options), len(output_options))+1, columnspan=2)

    window.mainloop()

    return selected_input, selected_output




# Phases de test

In [3]:
def start_midi_processing(input_port):
    """
    Enable the function "process_midi_messages" to be threaded
    As this function contains a "for loop" of midi messages input, it is endless. But we need to access also to the "root.mainloop()" event. So we thread this function    
    """
    midi_thread = threading.Thread(target=process_midi_messages, args=(input_port,))
    midi_thread.start()


def main(width=1600, height=400) -> None:
    """ 
    1) Open the input and output ports 
    2) Display the virtual piano keyboard window with Tkinter    
    3) Run the initialize_buttons function which initialized the different "game" modes of the program --> Click on one of those to play the mode
    4) Run the threading function to receive/send MIDI messages while the background virtual piano is running 

    """

    global canvas, root, output_port, input_port

    #Open ouput and input port 
    input_chosen, output_chosen = choose_ports()
    input_port, output_port = open_port(input_chosen, output_chosen)
    
    # Initialize the Tkinter window 
    root = tk.Tk()
    root.geometry("1600x600")

    # Load the piano background image and resize it
    background_image = Image.open("Images\piano.png")
    background_image = background_image.resize((width, height))
    background_image = ImageTk.PhotoImage(background_image)  

    # Create the canvas with the background piano image inside the Tkinter window
    canvas = tk.Canvas(root, width=width, height=height)
    canvas.pack()
    canvas.create_image(0, 0, image=background_image, anchor=tk.NW)


    #Intialize buttons 
    initialize_buttons()

    #Thread the MIDI messages tasks
    start_midi_processing(input_port)   


    root.mainloop()

In [ ]:
def initialize_buttons() -> None:
    """ 
    Initialize 3 buttons on the root tkinter window corresponding to the 3 game modes availbable: 
    - Note button: Play notes on the piano controller that have been requested 
    - Write button: Write the note with your keyboard that corresponds to the interval requested --> For example the request can be "Second major of C"
    - Chords button: Play the chords on the piano controller that have been requested

    Initialize the display_key_pressed variable to False, which indicates that while none of the game modes has been selected, you can play on your piano controller and get the 
    sound corresponding piano key but not the image of the piano key pressed on the piano background image 
    """
    
    global note_button, display_key_pressed, write_button, chords_button

    #Create a Button to request note
    note_button = tk.Button(root, text="NOTES", height=3, width=12, command=partial(button_clicked, request_note))
    note_button.place(x=0, y=410)

    #Create button that requests note appelation 
    write_button = tk.Button(root, text="WRITE", height=3, width=12, command=partial(button_clicked, write))
    write_button.place(x=0, y=470)

    #Create button that requests chords
    chords_button = tk.Button(root, text="CHORDS", height=3, width=12, command=partial(button_clicked, chords))
    chords_button.place(x=0, y=530)

    display_key_pressed = False #while the note_button is not pressed, we will just hear notes from controller, not the visual

In [ ]:
def button_clicked(button_name):
    """ 
    Activate the game mode you have chosen 

    Set quit_button_pressed to False indicating that you don't quit the game mode for now (for dict saving purpose)

    Initialize some sub-game modes buttons for certains game modes
    """

    global quit_button, next_button, activate_function, left_hand_button, right_hand_button, dict_to_save, quit_button_pressed, normal_button, reverse_button

    #Set quit_button to False, meaning that quit_button has not been pressed yet (for dict saving purpose)
    quit_button_pressed = False 

    activate_function = button_name
    
    #Delete temporary the 3 game modes buttons (still in the memory space)
    note_button.place_forget()
    write_button.place_forget()
    chords_button.place_forget()

    if activate_function is chords:
        #Create button for left hand exercise
        left_hand_button = tk.Button(root, text="LEFT HAND", height=5, width=15, command=partial(chords, hand_exercise="left_hand", activate_function_for_save="chords"))
        left_hand_button.place(x=550, y=450)

        #Create button for right hand exercise
        right_hand_button = tk.Button(root, text="RIGHT HAND", height=5, width=15, command=partial(chords, hand_exercise="right_hand", activate_function_for_save="chords"))
        right_hand_button.place(x=750, y=450)


    elif activate_function is write:
        #Create button for left hand exercise
        normal_button = tk.Button(root, text="NORMAL", height=5, width=15, command=partial(write, exercise_type="normal"))
        normal_button.place(x=550, y=450)

        #Create button for right hand exercise
        reverse_button = tk.Button(root, text="REVERSE", height=5, width=15, command=partial(write, exercise_type="reverse"))
        reverse_button.place(x=750, y=450)

    elif activate_function is request_note:
        


    else: #write and note functions

        if activate_function is request_note:
            dict_to_save = load_dict_to_save(activate_function_for_save="request_note")
        # elif activate_function is write:
        #     dict_to_save = load_dict_to_save(activate_function_for_save="write")
        quit_button = tk.Button(root, text="QUIT", height=3, width=12, command=quit)
        quit_button.place(x=350, y=500)

        next_button = tk.Button(root, text="NEXT", height=3, width=12, command=next)
        next_button.place(x=350, y=440)

        button_name()